In [7]:
import requests, json, time, datetime

In [1]:
matchids = {}
res = requests.get('http://canisback.com/matchId/matchlist_na1.json')
matchids = res.json()

In [2]:
# makes sure proper match ids were retrieved
print(len(matchids))

10000


In [4]:
apikey = #API KEY GOES HERE
matches = []

In [19]:
skipped_matchids = []
for i in range(1001,10000):
    try:
        #gets data for game
        res = requests.get(f'https://na1.api.riotgames.com/lol/match/v4/matches/{matchids[i]}?api_key={apikey}')
        match = res.json()
        if res.status_code == 200: 
            matches.append(res.json())
        else:
            print(f'ERROR message = {res.json()} Occured on gameid={matchids[i]}, index={i}')
            skipped_matchids.append(i)
    except requests.exceptions.RequestException as e:  
        raise SystemExit(e)
    if i%20 == 0: 
        time.sleep(1)
    if i%100 == 0:
        print(f'At {datetime.datetime.now()} we have {len(matches)} matches')
        time.sleep(120)   
    
print(f'Done! # of matches: {len(matches)}')

ERROR message = {'status': {'message': 'Gateway timeout', 'status_code': 504}} Occured on gameid=3514436049, index=1030
At 2020-08-01 13:09:08.130815 we have 1100 matches
At 2020-08-01 13:11:28.998873 we have 1200 matches
ERROR message = {'status': {'message': 'Gateway timeout', 'status_code': 504}} Occured on gameid=3514438014, index=1238
At 2020-08-01 13:13:49.437272 we have 1299 matches
At 2020-08-01 13:16:12.747653 we have 1399 matches
At 2020-08-01 13:18:34.331219 we have 1499 matches
ERROR message = {'status': {'message': 'Gateway timeout', 'status_code': 504}} Occured on gameid=3514443730, index=1516
At 2020-08-01 13:20:54.913751 we have 1598 matches
At 2020-08-01 13:23:16.497651 we have 1698 matches
At 2020-08-01 13:25:38.662531 we have 1798 matches
ERROR message = {'status': {'message': 'Gateway timeout', 'status_code': 504}} Occured on gameid=3514447170, index=1868
At 2020-08-01 13:27:59.699179 we have 1897 matches
ERROR message = {'status': {'message': 'Gateway timeout', 'st

In [20]:
for i in skipped_matchids:
    try:
        #gets data for game
        res = requests.get(f'https://na1.api.riotgames.com/lol/match/v4/matches/{matchids[i]}?api_key={apikey}')
        match = res.json()
        if res.status_code == 200: 
            matches.append(res.json())
        else:
            print(f'ERROR message = {res.json()} Occured on gameid={matchids[i]}, index={i}')
            skipped_matchids.append(i)
    except requests.exceptions.RequestException as e:  
        raise SystemExit(e)
    if i%20 == 0: 
        time.sleep(1)
    if i%100 == 0:
        print(f'At {datetime.datetime.now()} we have {len(matches)} matches')
        time.sleep(120) 

In [21]:
print(len(matches))

10000


In [22]:
count = 0
for m in matches:
    if m['queueId'] == 420:
        count+=1
print(count)

10000


In [23]:
simplemapping = {
    ('MIDDLE', 'SOLO'): 2, #MIDDLE
    ('TOP', 'SOLO'): 0, #TOP
    ('JUNGLE', 'NONE'): 1, #JUNGLE
    ('BOTTOM','DUO_CARRY'): 3, #BOTTOM
    ('BOTTOM', 'DUO_SUPPORT'): 4 #SUPPORT
}

0

In [24]:
data = []

for match in matches:  
    #x_vector will contain the features for the training example:
    # champion id's in the following order
    # [ top-1 jg-1 mid-1 adc-1 sup-1 top-2 jg-2 mid-2 adc-2 sup-2 ]
    # following the format role-teamnumber
    x_vector = [None] * 10
    bogus = False
    for player in match['participants']:
        try:
            if player['participantId'] < 6: #if player is in team 1
                index = simplemapping[(player['timeline']['lane'], player['timeline']['role'])]
                x_vector[index] = player['championId']
            else: #if player is in team 2
                index = simplemapping[(player['timeline']['lane'], player['timeline']['role'])] + 5
                x_vector[index] = player['championId']
        except KeyError: 
            bogus = True 
    
    if not bogus and None not in x_vector:      
        x_vector.append(1 if match['teams'][0]['win'] == 'Win' else 0)
        data.append(x_vector)

In [25]:
print(len(data))
print(data)

5172
[[19, 121, 3, 67, 12, 157, 48, 69, 29, 117, 0], [23, 121, 101, 518, 111, 517, 154, 39, 81, 555, 0], [86, 875, 131, 22, 412, 84, 141, 14, 101, 122, 0], [82, 121, 45, 51, 555, 114, 9, 55, 67, 53, 0], [75, 254, 25, 202, 117, 54, 120, 142, 51, 235, 0], [876, 121, 7, 29, 16, 122, 141, 134, 222, 63, 0], [92, 72, 55, 202, 412, 106, 427, 1, 51, 25, 1], [57, 77, 4, 18, 89, 98, 2, 25, 222, 412, 0], [58, 876, 69, 222, 53, 17, 141, 518, 18, 235, 1], [63, 19, 69, 51, 497, 23, 76, 245, 67, 235, 1], [58, 9, 18, 81, 63, 86, 28, 1, 145, 26, 1], [14, 254, 3, 110, 26, 106, 20, 61, 145, 99, 1], [48, 28, 157, 498, 63, 86, 11, 90, 202, 12, 0], [10, 876, 105, 67, 117, 39, 20, 103, 18, 3, 0], [164, 33, 157, 67, 53, 24, 77, 58, 498, 44, 1], [875, 133, 112, 51, 40, 107, 20, 85, 119, 117, 0], [82, 35, 80, 51, 89, 24, 106, 34, 22, 235, 1], [31, 102, 131, 21, 16, 122, 11, 61, 498, 497, 0], [86, 2, 1, 202, 235, 420, 24, 84, 222, 25, 0], [75, 33, 45, 51, 99, 5, 11, 105, 81, 37, 1], [86, 64, 105, 81, 40, 54, 11,

In [26]:
data_json = {'data':data}

with open('data.json', 'w') as outfile:
    json.dump(data_json, outfile)